In [1]:
import torch
import torch.nn as nn
# !pip install easyfsl
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import Omniglot
from torchvision.models import resnet18
from tqdm import tqdm

from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average

class PrototypicalNetworks_dynamic_query(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks_dynamic_query, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        num_classes=len(list(support_labels))
        prototypes = []
        for class_label in range(num_classes):
            # print(type(support_images[class_label]),torch.cat(support_images[class_label].shape)
            support_images_eachclass=(support_images[class_label])
            # print("kk", support_images_eachclass.shape)
            class_features = self.backbone(support_images_eachclass.to(device))  # Select features for the current class
            class_prototype = class_features.mean(dim=0)  # Compute the mean along the batch dimension
            prototypes.append(class_prototype)
        prototypes = torch.stack(prototypes)


        distances=[]
        for each_query_class in query_images:

          query_features = self.backbone(each_query_class)

          # Compute the distance between query features and prototypes
          distance = torch.cdist(query_features, prototypes)
          distances.append(-distance)

        # print(distances.shape)

        return distances


In [2]:
root_path = '/home/asufian/Desktop/output_olchiki/code/asamese'

In [3]:
import os
import random
from collections import defaultdict
from PIL import Image, ImageOps
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class PrototypicalOmniglotDataset(Dataset):
    def __init__(self, root, num_classes=1623, n_shot=5, n_query=10, transform=None):
        self.root = root
        self.num_classes = num_classes
        self.n_shot = n_shot
        self.n_query = n_query
        self.transform = transform
        self.samples_by_label = defaultdict(list)
        self.all_imgs = {}
        self.classes = []

        # Common image file extensions
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff', '.webp']

        # Organize data into classes and image paths
        for alphabet in os.listdir(self.root):
            alphabet_path = os.path.join(self.root, alphabet)
            if not os.path.isdir(alphabet_path):
                continue

            for char_class in os.listdir(alphabet_path):
                char_class_path = os.path.join(alphabet_path, char_class)

                # Skip if the entry is not a directory
                if not os.path.isdir(char_class_path):
                    continue

                all_images = []
                for img_name in sorted(os.listdir(char_class_path)):
                    img_path = os.path.join(char_class_path, img_name)

                    # Check if it's a file with a recognized image extension
                    if os.path.isfile(img_path) and any(img_name.lower().endswith(ext) for ext in image_extensions):
                        all_images.append(img_path)

                if all_images:
                    char_class_name = f"{alphabet}_{char_class}"
                    self.samples_by_label[char_class_name] = list(range(len(all_images)))
                    self.all_imgs[char_class_name] = all_images
                    self.classes.append(char_class_name)
                    # print(char_class_name)

    def transform_image(self, raw_img):
        img = raw_img#ImageOps.invert(raw_img)
        if self.transform is not None:
            # print(self.transform)
            img = self.transform(img)
        # print(img.shape)
        return img

    def __getitem__(self, selected_classes,selected_supports):#
        # selected_classes = random.sample(self.classes, self.num_classes)
        class_indices = [self.samples_by_label[each_cls] for each_cls in selected_classes]
        # print("selected_classes: ",selected_classes)
        support_set = []
        query_set = []
        label_id = 0
        qs=[]
        ko=0
        ghj=[]
        for idx_set in class_indices:
            # Creating support set
            selected_support =selected_supports[ko]# random.sample(idx_set, self.n_shot)
            ko=ko+1
            ghj.append(selected_support)
            # print("selected_support images: ", ghj)
            support_images = [self.transform_image(Image.open(self.all_imgs[selected_classes[label_id]][each]).convert('L')) for each in selected_support]

            # Creating query set
            selected_query = [item for item in idx_set if item not in selected_support]
            query_images = [self.transform_image(Image.open(self.all_imgs[selected_classes[label_id]][each]).convert('L')) for each in selected_query]
            support_set.append((support_images, [label_id for _ in range(self.n_shot)]))
            # print("aaaaaaaaaa",label_id)
            query_set.append((query_images,  [label_id for _ in range(len(selected_query))]))

            label_id += 1
            # qs= [item for item in a for _ in range(self.n_query)]
        return support_set, query_set

    def __len__(self):
        return 10

# Set up the transformations
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
])

# Set up the SiameseOmniglotDataset
# Training == evaluation
# root_path = './assamese'
num_classes = 5  # Set the desired number of classes per episode
n_shot = 5
n_query = 10

PrototypicalOmniglotDatasetLoader = PrototypicalOmniglotDataset(root=root_path, num_classes=num_classes, n_shot=n_shot, n_query=n_query, transform=transform)


In [4]:
import torch
import torch.nn as nn

class PrototypicalNetworks_dynamic_query(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks_dynamic_query, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        """
        Predict query labels using labeled support images.
        """
        num_classes=len(list(support_labels))
        prototypes = []
        for class_label in range(num_classes):
            # print(type(support_images[class_label]),torch.cat(support_images[class_label].shape)
            support_images_eachclass=(support_images[class_label])
            # print("kk", support_images_eachclass.shape)
            class_features = self.backbone(support_images_eachclass.to(device))  # Select features for the current class
            class_prototype = class_features.mean(dim=0)  # Compute the mean along the batch dimension
            prototypes.append(class_prototype)
        prototypes = torch.stack(prototypes)


        distances=[]
        for each_query_class in query_images:

          query_features = self.backbone(each_query_class)

          # Compute the distance between query features and prototypes
          distance = torch.cdist(query_features, prototypes)
          distances.append(-distance)

        # print(distances.shape)

        return distances


In [5]:
class PrototypicalNetworks33(nn.Module):
    def __init__(self, backbone: nn.Module):
        super(PrototypicalNetworks33, self).__init__()
        self.backbone = backbone

    def forward(
        self,
        support_images,
        support_labels,
        query_images: torch.Tensor,
    ) -> torch.Tensor:
        num_classes = len(support_labels)
        prototypes = []
        for class_label in range(num_classes):
            support_images_eachclass = support_images[class_label]
            class_features = torch.stack([self.backbone(each.unsqueeze(0)) for each in support_images_eachclass])
            prototype_bymean = class_features.mean(dim=0)
            # print(prototype_bymean.shape)
            prototypes.append(prototype_bymean)

        query_features = torch.stack([self.backbone(each.unsqueeze(0)) for each in query_images])
        # print(query_features.shape,query_features[0].shape,torch.stack(prototypes).shape,torch.stack(prototypes)[0].shape)
        # distances = -torch.cdist(query_features, torch.stack(prototypes))
        distances = []  # Initialize list to store distances
        prototypes=torch.stack(prototypes)
        for each_query_feature in query_features:
            d = torch.cdist(prototypes, each_query_feature)  # Compute distances
            distances.append(d.squeeze())  # Remove singleton dimension and store
            # print(d.shape)  # Output: [5, 1], shape of each distance matrix

        distances = torch.stack(distances)

        return -distances


In [39]:
import pandas as pd
import os
import csv
def save_in_csv(cls,flnm,conf_matrix, acciuracy,precision_overall, recall_overall, f_betas):
  with open(flnm, 'w', newline='') as csvfile:
      writer = csv.writer(csvfile)
      headers=['Accuracy', 'Precision', 'Recall']
      beta=-5
      values=[]
      values.append(acciuracy)
      values.append(precision_overall)
      values.append(recall_overall)
      for each in f_betas:
              headers.append('F-betas (' +str(beta)+' )')
              beta=beta+1
              values.append(each)

    # Write confusion matrix
      writer.writerow(['used Class:'])
      writer.writerows([cls])
      writer.writerow([])
      writer.writerow([])
      writer.writerow(['Confusion Matrix'])
      writer.writerows(conf_matrix)
      writer.writerow([])
      writer.writerow([])  # Add an empty row for separation

      # Write metrics

      writer.writerow(headers)
      writer.writerow(values)


  # print("Metrics have been written to 'metrics.csv' file.")
def save_in_text_file(msg,cls, flnm, conf_matrix, accuracy, precision_overall, recall_overall, f_betas):
    with open(flnm, 'a') as textfile:
        textfile.write(str("\n\n\n                                 "+msg))
        textfile.write("\n\n\n")
        textfile.write("used Class:\n")
        textfile.write(str(cls) + "\n\n")

        textfile.write("Confusion Matrix:\n")
        for row in conf_matrix:
            textfile.write(' '.join([str(elem) for elem in row]) + "\n")
        textfile.write("\n")

        textfile.write("Metrics:\n")
        textfile.write("Accuracy: {}\n".format(accuracy))
        textfile.write("Precision: {}\n".format(precision_overall))
        textfile.write("Recall: {}\n".format(recall_overall))

        beta = -5
        for f_beta in f_betas:
            textfile.write("F-betas ({}): {}\n".format(beta, f_beta))
            beta += 1

    # print("Metrics have been written to '{}' file.".format(flnm))

def save_in_excel_final(msg, cls, flnm, conf_matrix, accuracy, precision_overall, recall_overall):
    data = {
        "Used Class": [cls],
        "Used support images idx": [msg],
        "Confusion Matrix": [conf_matrix],
        "Accuracy": [accuracy],
        "Precision": [precision_overall],
        "Recall": [recall_overall]
    }
    # for i, f_beta in enumerate(f_betas):
    #     data[f"F-beta ({i - 5})"] = [f_beta]

    df_new = pd.DataFrame(data)
    df_new = df_new.transpose()  # Transpose the DataFrame

    if os.path.exists(flnm):
        df_existing = pd.read_excel(flnm, index_col=0)  # Read existing file
        df_combined = pd.concat([df_existing, df_new], axis=1)  # Concatenate along columns
        df_combined.to_excel(flnm)  # Write to Excel
    else:
        df_new.to_excel(flnm)  # Write new DataFrame to Excel

    # print("Metrics have been written to '{}' file.".format(flnm))


In [40]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch
import torch.nn as nn
import torch.hub

# Define a modified version of DenseNet169 with dropout after each convolutional layer
class DenseNet169WithDropout(nn.Module):
    def __init__(self,modell, pretrained=False, dr=0.4):
        super(DenseNet169WithDropout, self).__init__()
        self.densenet169 = torch.hub.load('pytorch/vision:v0.10.0', modell, pretrained=pretrained)
        self.dropout = nn.Dropout(p=dr)  # Dropout with probability 0.5

        # Modify each dense block to include dropout after each convolutional layer
        for name, module in self.densenet169.features.named_children():
            if isinstance(module, nn.Sequential):
                for sub_name, sub_module in module.named_children():
                    if isinstance(sub_module, nn.Conv2d):
                        setattr(module, sub_name, nn.Sequential(sub_module, self.dropout))

    def forward(self, x):
        return self.densenet169(x)

# Instantiate the modified DenseNet169 model with dropout
# convolutional_network_with_dropout = DenseNet169WithDropout(modell='resnet18',pretrained=False,dr=0.4)

# Example usage:
# output = convolutional_network_with_dropout(input_tensor)

class ResNet18WithDropout(nn.Module):
    def __init__(self, pretrained=False, dr=0.4):
        super(ResNet18WithDropout, self).__init__()
        self.resnet18 = models.resnet18(pretrained=pretrained)
        self.dropout = nn.Dropout(p=dr)

        # Modify each residual block to include dropout after each convolutional layer
        for name, module in self.resnet18.named_children():
            if isinstance(module, nn.Sequential):
                for sub_name, sub_module in module.named_children():
                    if isinstance(sub_module, nn.Conv2d):
                        setattr(module, sub_name, nn.Sequential(sub_module, self.dropout))

    def forward(self, x):
        return self.resnet18(x)

# Instantiate the modified ResNet18 model with dropout
# convolutional_network_with_dropout = ResNet18WithDropout(pretrained=False, dr=0.4)


In [41]:
clss_5=[
    ['assamese1_BXG', 'assamese1_NAA', 'assamese1_KHYA', 'assamese1_CBN', 'assamese1_DHA'],
      ['assamese1_DRA', 'assamese1_OI', 'assamese1_CA', 'assamese1_JHA', 'assamese1_MA'],
     ['assamese1_KHA', 'assamese1_CARI', 'assamese1_CA', 'assamese1_CAY', 'assamese1_TA'],
    ['assamese1_CA', 'assamese1_MNA', 'assamese1_TR', 'assamese1_CAY', 'assamese1_KHYA'],
    ['assamese1_NA', 'assamese1_KTA', 'assamese1_MDA', 'assamese1_MDHA', 'assamese1_MTA'],
     ['assamese1_OI', 'assamese1_MXA', 'assamese1_ATH', 'assamese1_DA', 'assamese1_BHA'],
     ['assamese1_PAC', 'assamese1_ANSR', 'assamese1_E', 'assamese1_TR', 'assamese1_BHA'],
     ['assamese1_CA', 'assamese1_CARI', 'assamese1_A', 'assamese1_AYA', 'assamese1_NIYA'],
    ['assamese1_MDHA', 'assamese1_BHA', 'assamese1_MTA', 'assamese1_EK', 'assamese1_GHA'],
    ['assamese1_ATH', 'assamese1_CAY', 'assamese1_BHA', 'assamese1_TXA', 'assamese1_MDHA'],
    ['assamese1_OU', 'assamese1_AE', 'assamese1_BXG', 'assamese1_JA', 'assamese1_MDHA'],
    ['assamese1_MXA', 'assamese1_ANSR', 'assamese1_CA', 'assamese1_MTA', 'assamese1_THA'],
     ['assamese1_HA', 'assamese1_KHA', 'assamese1_OI', 'assamese1_NG', 'assamese1_MNA'],
    ['assamese1_NG', 'assamese1_MDHA', 'assamese1_KA', 'assamese1_THA', 'assamese1_BA'],
    ['assamese1_UU', 'assamese1_EK', 'assamese1_SUNYA', 'assamese1_CA', 'assamese1_TR'],

     ]

clss_support_imagesss_1_shot=[
    [[35], [11], [7], [4], [12]],
    [[1], [15], [2], [23], [43]],
    [[20], [44], [5], [15], [37]],
    [[37], [33], [44], [6], [17]],
    [[27], [33], [9], [40], [40]],
    [[22], [6], [18], [37], [7]],
    [[9], [40], [34], [8], [1]],
    [[9], [5], [10], [2], [9]],
    [[17], [10], [27], [39], [6]],
    [[28], [38], [38], [26], [43]],
    [[35], [2], [12], [27], [38]],
    [[12], [11], [2], [3], [19]],
     [[39], [1], [2], [2], [25]],
    [[19], [0], [10], [19], [3]],
    [[37], [1], [1], [7], [9]]







]


clss_support_imagesss_5_shot=[[[35, 8, 42, 10, 39], [11, 28, 8, 10, 26], [7, 10, 6, 21, 36], [4, 9, 24, 11, 1], [12, 34, 2, 40, 18]], [[1, 8, 13, 24, 7], [15, 42, 34, 19, 13], [2, 8, 1, 19, 40], [23, 14, 17, 11, 28], [43, 25, 2, 13, 21]], [[20, 30, 9, 22, 13], [44, 21, 13, 20, 36], [5, 20, 15, 41, 19], [15, 40, 33, 25, 14], [37, 15, 27, 7, 41]], [[37, 31, 39, 42, 15], [33, 15, 34, 19, 3], [44, 0, 24, 25, 26], [6, 17, 8, 15, 9], [17, 13, 15, 5, 44]], [[27, 17, 4, 12, 42], [33, 0, 30, 31, 15], [9, 11, 39, 41, 34], [40, 43, 9, 32, 41], [40, 3, 18, 38, 4]], [[22, 8, 10, 25, 15], [6, 28, 5, 3, 31], [18, 32, 23, 38, 4], [37, 1, 24, 13, 30], [7, 11, 38, 32, 5]], [[9, 28, 6, 5, 18], [40, 29, 12, 5, 39], [34, 22, 25, 23, 8], [8, 32, 41, 2, 30], [1, 18, 41, 39, 40]], [[9, 24, 28, 5, 14], [5, 28, 36, 4, 27], [10, 24, 17, 12, 14], [2, 17, 20, 9, 7], [9, 29, 3, 28, 37]], [[17, 6, 31, 39, 36], [10, 34, 8, 37, 14], [27, 38, 14, 15, 28], [39, 5, 38, 4, 2], [6, 26, 27, 31, 43]], [[28, 12, 25, 38, 1], [38, 15, 37, 43, 29], [38, 29, 2, 4, 11], [26, 4, 27, 13, 14], [43, 20, 11, 38, 16]], [[35, 18, 1, 11, 17], [2, 18, 25, 1, 12], [12, 25, 18, 24, 40], [27, 16, 30, 25, 35], [38, 39, 9, 22, 24]], [[12, 2, 27, 4, 17], [11, 16, 36, 20, 8], [2, 28, 17, 32, 26], [3, 20, 28, 22, 4], [19, 1, 9, 0, 11]], [[39, 16, 19, 32, 34], [1, 9, 20, 0, 8], [2, 22, 20, 43, 21], [2, 13, 44, 21, 19], [25, 1, 41, 3, 33]], [[19, 40, 42, 43, 27], [0, 1, 30, 12, 40], [10, 43, 15, 2, 20], [19, 43, 38, 16, 33], [3, 30, 17, 1, 41]], [[37, 11, 25, 28, 35], [1, 3, 35, 4, 33], [1, 31, 25, 32, 13], [7, 3, 10, 11, 35], [9, 19, 1, 22, 8]]]

clss_support_imagesss_10_shot=[[[35, 8, 42, 10, 39, 30, 15, 11, 44, 7], [11, 28, 8, 10, 26, 16, 12, 7, 44, 37], [7, 10, 6, 21, 36, 24, 22, 38, 9, 39], [4, 9, 24, 11, 1, 32, 13, 28, 30, 17], [12, 34, 2, 40, 18, 38, 1, 14, 22, 36]], [[1, 8, 13, 24, 7, 12, 44, 29, 22, 23], [15, 42, 34, 19, 13, 30, 0, 23, 1, 9], [2, 8, 1, 19, 40, 44, 32, 26, 27, 9], [23, 14, 17, 11, 28, 16, 24, 35, 6, 34], [43, 25, 2, 13, 21, 17, 20, 38, 24, 35]], [[20, 30, 9, 22, 13, 36, 18, 21, 3, 37], [44, 21, 13, 20, 36, 33, 22, 0, 34, 12], [5, 20, 15, 41, 19, 10, 44, 12, 28, 26], [15, 40, 33, 25, 14, 44, 19, 32, 24, 6], [37, 15, 27, 7, 41, 9, 43, 33, 35, 23]], [[37, 31, 39, 42, 15, 11, 25, 1, 36, 21], [33, 15, 34, 19, 3, 28, 25, 29, 40, 37], [44, 0, 24, 25, 26, 6, 4, 19, 12, 14], [6, 17, 8, 15, 9, 25, 27, 29, 7, 0], [17, 13, 15, 5, 44, 33, 8, 6, 34, 31]], [[27, 17, 4, 12, 42, 11, 6, 13, 33, 3], [33, 0, 30, 31, 15, 34, 17, 14, 36, 20], [9, 11, 39, 41, 34, 10, 8, 15, 31, 4], [40, 43, 9, 32, 41, 14, 35, 10, 20, 23], [40, 3, 18, 38, 4, 41, 22, 29, 15, 27]], [[22, 8, 10, 25, 15, 14, 29, 2, 41, 24], [6, 28, 5, 3, 31, 2, 32, 40, 16, 13], [18, 32, 23, 38, 4, 2, 34, 22, 16, 36], [37, 1, 24, 13, 30, 5, 36, 27, 39, 35], [7, 11, 38, 32, 5, 15, 8, 41, 10, 2]], [[9, 28, 6, 5, 18, 44, 3, 8, 2, 34], [40, 29, 12, 5, 39, 18, 9, 43, 19, 28], [34, 22, 25, 23, 8, 18, 17, 24, 19, 6], [8, 32, 41, 2, 30, 5, 9, 17, 44, 1], [1, 18, 41, 39, 40, 7, 0, 25, 24, 15]], [[9, 24, 28, 5, 14, 16, 25, 22, 32, 37], [5, 28, 36, 4, 27, 18, 32, 2, 10, 31], [10, 24, 17, 12, 14, 41, 4, 32, 42, 37], [2, 17, 20, 9, 7, 31, 44, 38, 1, 22], [9, 29, 3, 28, 37, 17, 41, 13, 21, 44]], [[17, 6, 31, 39, 36, 8, 11, 24, 10, 7], [10, 34, 8, 37, 14, 16, 29, 44, 31, 33], [27, 38, 14, 15, 28, 23, 7, 42, 35, 41], [39, 5, 38, 4, 2, 42, 6, 22, 21, 9], [6, 26, 27, 31, 43, 41, 10, 12, 36, 32]], [[28, 12, 25, 38, 1, 41, 15, 42, 33, 20], [38, 15, 37, 43, 29, 44, 41, 39, 12, 21], [38, 29, 2, 4, 11, 43, 33, 24, 7, 21], [26, 4, 27, 13, 14, 16, 28, 0, 3, 36], [43, 20, 11, 38, 16, 14, 27, 13, 19, 0]], [[35, 18, 1, 11, 17, 19, 31, 16, 7, 15], [2, 18, 25, 1, 12, 16, 9, 23, 11, 44], [12, 25, 18, 24, 40, 35, 17, 44, 28, 22], [27, 16, 30, 25, 35, 18, 26, 42, 33, 40], [38, 39, 9, 22, 24, 18, 29, 21, 1, 7]], [[12, 2, 27, 4, 17, 42, 37, 3, 18, 10], [11, 16, 36, 20, 8, 4, 42, 28, 44, 7], [2, 28, 17, 32, 26, 3, 37, 16, 10, 14], [3, 20, 28, 22, 4, 43, 17, 9, 0, 11], [19, 1, 9, 0, 11, 7, 27, 15, 18, 6]], [[39, 16, 19, 32, 34, 31, 13, 4, 37, 6], [1, 9, 20, 0, 8, 29, 4, 27, 26, 13], [2, 22, 20, 43, 21, 4, 15, 13, 14, 10], [2, 13, 44, 21, 19, 5, 14, 23, 43, 26], [25, 1, 41, 3, 33, 16, 9, 37, 7, 30]], [[19, 40, 42, 43, 27, 30, 3, 10, 21, 28], [0, 1, 30, 12, 40, 33, 11, 22, 44, 27], [10, 43, 15, 2, 20, 32, 14, 30, 11, 42], [19, 43, 38, 16, 33, 17, 35, 14, 10, 5], [3, 30, 17, 1, 41, 29, 40, 43, 8, 31]], [[37, 11, 25, 28, 35, 13, 5, 9, 31, 6], [1, 3, 35, 4, 33, 8, 41, 26, 12, 37], [1, 31, 25, 32, 13, 14, 22, 9, 26, 19], [7, 3, 10, 11, 35, 13, 41, 26, 43, 20], [9, 19, 1, 22, 8, 17, 40, 3, 32, 14]]]


In [42]:
!ls '/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_1'

model_B_olchiki_1-shot_dens_0.pth   model_B_olchiki_1-shot_dens_21.pth
model_B_olchiki_1-shot_dens_10.pth  model_B_olchiki_1-shot_dens_28.pth
model_B_olchiki_1-shot_dens_14.pth  model_B_olchiki_1-shot_dens_7.pth


In [43]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")




convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_1/model_B_olchiki_1-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [44]:
# !ls '/home/asufian/Desktop/output_olchiki/'

In [45]:
criterion = nn.CrossEntropyLoss()

In [46]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import numpy as np
import numpy as np
import pandas as pd
# root_path = '/content/assamese'
num_classes = 5
n_shot = 1
n_query = 14
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformt_ugh = transforms.Compose([
    transforms.Resize((64, 64)),
    # transforms.RandomResizedCrop(64),  # Randomly crop the image and resize to 224x224
    # transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    # transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),  # Randomly adjust brightness, contrast, saturation, and hue
    # transforms.RandomAffine(degrees=0, translate=(0, 0), scale=(0.9, 1.1), shear=0),  # Random zoom (scaling)
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),  # Convert PIL Image to PyTorch Tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize the image with ImageNet statistics
])

print(f"Generated random values - num_classes: {num_classes}, n_shot: {n_shot}, n_query: {n_query}")

test_loader = PrototypicalOmniglotDataset(root=root_path, num_classes=num_classes, n_shot=n_shot, n_query=20, transform=transformt_ugh)

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    precision=0
    recall=0
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            # msg="\n\n       Way: "+str(len(clss_5[tlv_cls]))+"  Shot: "+str(len(clss_support_imagesss_1_shot[tlv_cls][0]))    +" \n\n"
            # msg=str(clss_5[tlv_cls])
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision=precision+precision_overall
            recall=recall+recall_overall
    print("---------------------->",ttlac/ttla,precision/ttla ,recall/ttla)
# %%%%%%%%%%%%%%%%%%%


pp='/home/asufian/Desktop/output_olchiki/faltu'






Generated random values - num_classes: 5, n_shot: 1, n_query: 14


In [47]:
import numpy as np

conf_matrix = np.array([[100, 0, 0, 0, 0],
                        [0, 100, 0, 0, 0],
                        [0, 0, 100, 0, 0],
                        [0, 0, 0, 100, 0],
                        [0, 0, 0, 0, 100]])

def calculate_metrics_get_per(msg, conf_matrix,acciuracy, cls, flnm):
    if conf_matrix.size == 0:
        raise ValueError("Confusion matrix is empty!")

    metrics = []
    for i in range(conf_matrix.shape[0]):
        tp = conf_matrix[i, i]
        fp = np.sum(conf_matrix[:, i]) - tp
        fn = np.sum(conf_matrix[i, :]) - tp

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics.append((precision, recall, f1))

    # Compute overall scores
    precision_overall = np.mean([m[0] for m in metrics])
    recall_overall = np.mean([m[1] for m in metrics])
    f1_overall = np.mean([m[2] for m in metrics])

    # Compute overall accuracy
    total_samples = np.sum(conf_matrix)
    correct_predictions = np.sum(np.diag(conf_matrix))
    accuracy_overall = correct_predictions / total_samples if total_samples > 0 else 0
    # print(accuracy_overall)

    # Save results (Assuming these functions exist)
    save_in_excel_final(msg, cls, flnm + ".xlsx", conf_matrix, accuracy_overall, precision_overall, recall_overall)
    # save_in_text_file(msg, cls, flnm + ".txt", conf_matrix, accuracy_overall, precision_overall, recall_overall, f_betas)

    return precision_overall, recall_overall, f1_overall


# precision, recall, f_beta = calculate_metrics(conf_matrix)
# print(f'Overall Metrics for beta={beta}:')
# print(f'Overall Precision: {precision:.4f}')
# print(f'Overall Recall: {recall:.4f}')
# print(f'Overall F-beta ({f_beta}): ')
# print()


In [48]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [49]:
def update_model_weights2(model_a, model_b, conv_ratio=0.8, fc_ratio=0.8, bias_ratio=0.8):
    """Updates model_a's weights using a weighted combination of model_a and model_b."""
    for name, param in model_a.named_parameters():
        if name in model_b.state_dict():
            weight_b = model_b.state_dict()[name]
            
            if 'weight' in name:
                # Separate conditions for convolutional and fully connected layers
                if 'conv' in name:  # Convolutional layer weights
                    # print('conv')
                    param.data = conv_ratio * weight_b + (1 - conv_ratio) * param.data
                elif 'fc' in name or 'linear' in name:  # Fully connected layer weights
                    # print('fc')
                    param.data = fc_ratio * weight_b + (1 - fc_ratio) * param.data
            
            elif 'bias' in name:
                param.data = bias_ratio * weight_b + (1 - bias_ratio) * param.data

In [50]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to get model accuracy
def get_model_accuracy(b_a_r, c_a_r, f_a_r):
    try:
        print(f"Testing b_a_r = {b_a_r:.4f}, c_a_r = {c_a_r:.4f}, f_a_r = {f_a_r:.4f}")  # Debugging Output

        convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
        convolutional_network_with_dropout.fc = nn.Flatten()
        xyz = convolutional_network_with_dropout
        M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
        M3.load_state_dict(torch.load('model_own_path.pt'))

        # Load Model M2 (Reference)
        convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
        convolutional_network_with_dropout.fc = nn.Flatten()
        xyz2 = convolutional_network_with_dropout
        M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
        M2.load_state_dict(torch.load('model_mu_path.pt'))

        # Apply bias adaptation to M3 using M2
        update_model_weights2(M3, M2, conv_ratio=c_a_r, fc_ratio=f_a_r, bias_ratio=b_a_r)
        
        # Evaluate accuracy
        accuracy = evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)
        
        if accuracy is None:
            raise ValueError("Accuracy computation failed. Returning worst case.")

        return accuracy  # Ensure accuracy is valid
    except Exception as e:
        print(f"Error encountered: {e}. Assigning worst-case accuracy of 0.")
        return 0.0  # Assign lowest accuracy to prevent optimization failures


In [51]:
root_path

'/home/asufian/Desktop/output_olchiki/code/asamese'

In [52]:
criterion = nn.CrossEntropyLoss()


In [53]:

# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5457564738064334
Evaluated Accuracy: 0.5458
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5036694159095448
Evaluated Accuracy: 0.5037
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.6081988295525765
Evaluated Accuracy: 0.6082
Testing b_a_r = 0.9715, c_a_r = 0.6498, f_a_r = 0.8477


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3821581149623051
Evaluated Accuracy: 0.3822
Testing b_a_r = 0.0196, c_a_r = 0.3149, f_a_r = 0.9835


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5987993177920656
Evaluated Accuracy: 0.5988

✅ Optimal Values:
   - b_a_r: 0.0200
   - c_a_r: 0.2915
   - f_a_r: 0.0062
📈 Highest Accuracy Achieved: 0.6085


In [22]:
# !mkdir '/home/asufian/Desktop/output_assamese/Densnet_resluts/amul'

mkdir: cannot create directory ‘/home/asufian/Desktop/output_assamese/Densnet_resluts/amul’: File exists


In [54]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/5_w_1_s_f__ff', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/5_w_1_s_f__ff", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/5_w_1_s_f__ff", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4727 ± 0.1013  0.5204 ± 0.0908  0.4727 ± 0.1013


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4361 ± 0.1289  0.4700 ± 0.1229  0.4362 ± 0.1289
---Accu-----pre----rec---------> 0.6085 ± 0.1197  0.6499 ± 0.1077  0.6085 ± 0.1197


In [20]:
# !mkdir '/home/asufian/Desktop/output_assamese/Densnet_resluts/amul'

In [21]:
!ls /home/asufian/Desktop/output_olchiki/code/olchiki/model_5/model_B_olchiki_1-shot_30.pth

model_B_olchiki_1-shot_20.pth  model_B_olchiki_1-shot_80.pth
model_B_olchiki_1-shot_30.pth  model_B_olchiki_1-shot_90.pth
model_B_olchiki_1-shot_50.pth


In [ ]:
########################  5   way    5  shot    #########################

In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_5/model_B_olchiki_5-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [56]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [57]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6820290840604349
Evaluated Accuracy: 0.6820
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6243605423468919
Evaluated Accuracy: 0.6244
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.639687358850638
Evaluated Accuracy: 0.6397
Testing b_a_r = 0.0294, c_a_r = 0.3195, f_a_r = 0.0607


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.7313608256873089
Evaluated Accuracy: 0.7314
Testing b_a_r = 0.0221, c_a_r = 0.2603, f_a_r = 0.0556


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.7313508254373026
Evaluated Accuracy: 0.7314

✅ Optimal Values:
   - b_a_r: 0.0186
   - c_a_r: 0.3041
   - f_a_r: 0.0365
📈 Highest Accuracy Achieved: 0.7394


In [58]:
# b_a_r: 
#    - c_a_r: 
#    - f_a_r: 

#  pt=20

In [59]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/5_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/5_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/5_w_5_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6287 ± 0.0554  0.6502 ± 0.0607  0.6286 ± 0.0553


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5210 ± 0.0681  0.5310 ± 0.0738  0.5210 ± 0.0681
---Accu-----pre----rec---------> 0.7394 ± 0.0623  0.7523 ± 0.0575  0.7394 ± 0.0623


In [33]:
!ls /home/asufian/Desktop/output_olchiki/code/olchiki/model_10

model_B_olchiki_1-shot_20.pth  model_B_olchiki_1-shot_70.pth
model_B_olchiki_1-shot_40.pth  model_B_olchiki_1-shot_90.pth
model_B_olchiki_1-shot_50.pth


In [60]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_10/model_B_olchiki_10-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [61]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_5)):
            support_set, query_set = data_loader.__getitem__(clss_5[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_5[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_5[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [62]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6853495546598994
Evaluated Accuracy: 0.6853
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6575551077275216
Evaluated Accuracy: 0.6576
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.7264946758222621
Evaluated Accuracy: 0.7265
Testing b_a_r = 0.9793, c_a_r = 0.9998, f_a_r = 0.6363


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6720659799970145
Evaluated Accuracy: 0.6721
Testing b_a_r = 0.9683, c_a_r = 0.2879, f_a_r = 0.0266


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.7436462904911181
Evaluated Accuracy: 0.7436

✅ Optimal Values:
   - b_a_r: 0.0042
   - c_a_r: 0.2752
   - f_a_r: 0.0399
📈 Highest Accuracy Achieved: 0.7520


In [63]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/5_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/5_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/5_w_10_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6305 ± 0.0457  0.6493 ± 0.0567  0.6303 ± 0.0456


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.6172 ± 0.0776  0.6232 ± 0.0824  0.6171 ± 0.0776
---Accu-----pre----rec---------> 0.7520 ± 0.0609  0.7598 ± 0.0575  0.7520 ± 0.0609


In [64]:
ls /home/asufian/Desktop/output_olchiki/lol

5_w_10_s.txt  5_w_10_s.xlsx


In [65]:
clss_8=[
       ['assamese1_DXA', 'assamese1_SUNYA', 'assamese1_KTA', 'assamese1_CARI', 'assamese1_AA', 'assamese1_ATH', 'assamese1_DUI', 'assamese1_MNA'],
     ['assamese1_BA', 'assamese1_OU', 'assamese1_NA', 'assamese1_TINI', 'assamese1_MDHA', 'assamese1_E', 'assamese1_A', 'assamese1_TXA'],
    ['assamese1_ATH', 'assamese1_CA', 'assamese1_NG', 'assamese1_EK', 'assamese1_ANSR', 'assamese1_AA', 'assamese1_KTA', 'assamese1_DHA'],
    ['assamese1_KA', 'assamese1_XAT', 'assamese1_E', 'assamese1_MA', 'assamese1_CCA', 'assamese1_MTA', 'assamese1_A', 'assamese1_AE'],
    ['assamese1_TINI', 'assamese1_KHYA', 'assamese1_NAA', 'assamese1_HA', 'assamese1_DUI', 'assamese1_CBN', 'assamese1_TR', 'assamese1_AE'],
     ['assamese1_DA', 'assamese1_SUNYA', 'assamese1_AJA', 'assamese1_ANSR', 'assamese1_PHA', 'assamese1_NAA', 'assamese1_MDHA', 'assamese1_NG'],
    ['assamese1_DUI', 'assamese1_E', 'assamese1_AYA', 'assamese1_MA', 'assamese1_THA', 'assamese1_MTA', 'assamese1_ATH', 'assamese1_NIYA'],
    ['assamese1_MA', 'assamese1_AYA', 'assamese1_NIYA', 'assamese1_KTA', 'assamese1_ATH', 'assamese1_PAC', 'assamese1_KHYA', 'assamese1_DUI'],
     ['assamese1_DXA', 'assamese1_CAY', 'assamese1_MNA', 'assamese1_ANSR', 'assamese1_AA', 'assamese1_TXA', 'assamese1_GA', 'assamese1_ATH'],
    ['assamese1_RA', 'assamese1_ATH', 'assamese1_CARI', 'assamese1_NG', 'assamese1_DA', 'assamese1_REE', 'assamese1_AJA', 'assamese1_A'],
    ['assamese1_KHA', 'assamese1_THA', 'assamese1_AA', 'assamese1_MNA', 'assamese1_SUNYA', 'assamese1_NIYA', 'assamese1_LA', 'assamese1_MDHA'],
     ['assamese1_KA', 'assamese1_AYA', 'assamese1_LA', 'assamese1_TINI', 'assamese1_MA', 'assamese1_KTA', 'assamese1_MDHA', 'assamese1_PA'],
    ['assamese1_DHRA', 'assamese1_CCA', 'assamese1_DXA', 'assamese1_TXA', 'assamese1_SUNYA', 'assamese1_ANSR', 'assamese1_HA', 'assamese1_OU'],
    ['assamese1_NG', 'assamese1_JHA', 'assamese1_GHA', 'assamese1_MTHA', 'assamese1_ANSR', 'assamese1_ATH', 'assamese1_PA', 'assamese1_MDHA'],
    ['assamese1_ATH', 'assamese1_CAY', 'assamese1_DUI', 'assamese1_AJA', 'assamese1_AA', 'assamese1_THA', 'assamese1_BHA', 'assamese1_HA'],
    ]

clss_support_imagesss_10_shot=[[[24, 30, 13, 26, 3, 2, 42, 12, 29, 33], [16, 31, 30, 15, 22, 33, 6, 41, 0, 12], [7, 31, 42, 1, 30, 35, 20, 39, 0, 3], [39, 41, 6, 34, 16, 36, 17, 1, 43, 44], [42, 5, 38, 41, 7, 37, 6, 1, 28, 2], [2, 10, 15, 24, 38, 5, 29, 18, 9, 32], [34, 38, 18, 5, 25, 14, 19, 29, 0, 2], [7, 13, 26, 43, 4, 41, 8, 2, 18, 30]], [[14, 31, 35, 11, 44, 40, 9, 20, 12, 24], [43, 35, 29, 0, 10, 24, 34, 27, 11, 38], [40, 42, 34, 41, 12, 31, 15, 10, 44, 5], [21, 29, 4, 10, 19, 34, 5, 2, 41, 0], [18, 25, 11, 5, 43, 8, 19, 13, 42, 39], [18, 41, 2, 3, 33, 6, 21, 36, 32, 23], [44, 1, 16, 29, 35, 26, 19, 18, 36, 37], [36, 12, 22, 44, 29, 1, 2, 39, 34, 35]], [[10, 26, 40, 36, 38, 16, 2, 39, 6, 11], [41, 23, 19, 24, 6, 34, 13, 35, 22, 44], [29, 0, 40, 32, 41, 3, 11, 34, 39, 13], [40, 6, 0, 24, 14, 41, 37, 25, 27, 36], [36, 35, 32, 42, 7, 19, 33, 13, 17, 22], [13, 7, 12, 44, 19, 39, 40, 37, 29, 36], [17, 19, 23, 8, 31, 18, 32, 26, 22, 34], [19, 17, 0, 13, 28, 32, 20, 22, 1, 24]], [[23, 2, 25, 26, 12, 4, 34, 6, 13, 19], [15, 14, 24, 1, 26, 35, 28, 5, 13, 34], [28, 30, 25, 41, 33, 37, 20, 4, 3, 15], [39, 13, 41, 6, 35, 38, 29, 20, 14, 19], [38, 30, 10, 12, 3, 40, 25, 34, 0, 27], [0, 16, 27, 13, 40, 43, 41, 1, 39, 9], [27, 42, 35, 38, 40, 20, 15, 2, 26, 6], [14, 22, 29, 10, 37, 33, 28, 27, 43, 13]], [[7, 34, 0, 37, 21, 33, 44, 2, 24, 26], [28, 21, 36, 32, 33, 9, 1, 13, 5, 29], [28, 30, 18, 8, 27, 13, 22, 36, 33, 4], [32, 22, 36, 42, 0, 11, 39, 7, 37, 38], [16, 11, 40, 14, 38, 7, 33, 28, 26, 20], [36, 44, 43, 12, 38, 9, 13, 34, 30, 5], [22, 30, 25, 4, 44, 40, 16, 21, 36, 18], [11, 12, 26, 9, 29, 20, 37, 19, 30, 0]], [[37, 8, 41, 16, 27, 2, 13, 39, 38, 24], [10, 43, 42, 5, 8, 16, 34, 23, 9, 1], [13, 11, 41, 44, 32, 17, 3, 9, 34, 28], [36, 1, 4, 18, 24, 42, 22, 17, 10, 15], [37, 35, 40, 12, 41, 28, 22, 18, 4, 17], [8, 33, 15, 1, 3, 0, 17, 44, 20, 12], [4, 19, 20, 24, 12, 15, 6, 14, 10, 17], [21, 13, 38, 24, 9, 26, 22, 23, 41, 6]], [[42, 30, 16, 39, 41, 8, 23, 37, 9, 12], [13, 3, 29, 10, 25, 38, 40, 14, 11, 5], [36, 26, 14, 27, 6, 0, 38, 18, 42, 44], [20, 38, 33, 6, 36, 8, 40, 4, 18, 44], [30, 26, 38, 24, 8, 14, 0, 6, 19, 37], [8, 27, 34, 17, 43, 23, 11, 30, 21, 44], [12, 36, 35, 25, 39, 43, 29, 44, 10, 21], [12, 7, 22, 0, 36, 8, 28, 29, 15, 26]], [[40, 8, 43, 38, 23, 3, 24, 12, 25, 22], [27, 20, 14, 31, 9, 11, 42, 23, 5, 38], [22, 12, 27, 1, 17, 38, 41, 21, 13, 2], [6, 16, 8, 42, 3, 7, 35, 22, 44, 21], [2, 12, 30, 44, 4, 10, 38, 6, 36, 42], [32, 17, 21, 38, 36, 43, 10, 29, 20, 4], [15, 25, 43, 10, 5, 35, 19, 20, 12, 0], [2, 13, 15, 16, 43, 8, 9, 5, 41, 24]], [[1, 26, 39, 8, 5, 22, 16, 38, 29, 19], [12, 11, 43, 28, 40, 6, 38, 24, 8, 16], [6, 14, 20, 2, 31, 28, 35, 19, 34, 33], [29, 23, 1, 25, 3, 31, 18, 27, 9, 12], [22, 42, 35, 3, 37, 6, 31, 38, 7, 41], [14, 23, 37, 44, 30, 18, 24, 32, 4, 25], [26, 10, 13, 32, 25, 36, 7, 29, 41, 15], [31, 16, 35, 9, 6, 11, 3, 41, 2, 25]], [[25, 37, 5, 10, 33, 36, 3, 19, 43, 29], [11, 2, 23, 29, 3, 43, 37, 21, 44, 7], [40, 17, 11, 14, 31, 38, 7, 3, 4, 42], [39, 29, 16, 19, 6, 37, 4, 23, 5, 15], [22, 5, 34, 36, 13, 44, 29, 16, 33, 25], [20, 15, 17, 38, 33, 29, 30, 13, 1, 8], [42, 44, 22, 0, 29, 19, 6, 13, 25, 1], [4, 38, 3, 35, 19, 34, 20, 29, 42, 6]], [[7, 29, 19, 4, 40, 6, 30, 12, 41, 31], [38, 3, 29, 17, 14, 28, 37, 33, 36, 40], [22, 30, 11, 17, 10, 32, 39, 12, 2, 6], [29, 17, 9, 44, 20, 21, 12, 19, 25, 4], [5, 21, 37, 23, 41, 40, 38, 16, 12, 25], [23, 8, 21, 15, 43, 44, 7, 12, 26, 16], [6, 4, 26, 38, 10, 20, 28, 13, 0, 39], [27, 36, 2, 1, 28, 4, 5, 24, 13, 33]], [[42, 1, 20, 12, 34, 24, 8, 26, 15, 18], [40, 41, 43, 11, 24, 27, 10, 21, 29, 38], [16, 34, 19, 39, 7, 22, 29, 5, 35, 17], [24, 34, 37, 16, 35, 31, 20, 1, 42, 38], [33, 2, 16, 7, 3, 4, 39, 20, 11, 35], [43, 17, 24, 14, 40, 5, 29, 39, 13, 38], [6, 3, 21, 27, 19, 10, 44, 9, 30, 26], [16, 3, 25, 22, 38, 42, 27, 23, 20, 18]], [[40, 30, 1, 23, 41, 0, 12, 25, 43, 35], [18, 7, 20, 31, 21, 30, 43, 11, 42, 12], [25, 14, 2, 28, 24, 3, 39, 9, 36, 33], [23, 14, 32, 35, 17, 20, 29, 5, 30, 16], [19, 17, 34, 18, 20, 39, 7, 24, 29, 35], [4, 39, 15, 25, 44, 7, 6, 23, 29, 37], [39, 34, 23, 13, 2, 6, 42, 25, 7, 1], [7, 9, 19, 34, 39, 28, 4, 41, 29, 33]], [[1, 29, 19, 21, 43, 41, 36, 15, 14, 18], [36, 33, 23, 31, 19, 25, 11, 32, 21, 28], [43, 26, 42, 29, 2, 13, 39, 41, 33, 19], [36, 27, 34, 20, 43, 10, 33, 7, 37, 28], [33, 25, 9, 16, 2, 26, 39, 41, 11, 8], [16, 39, 40, 12, 13, 0, 1, 27, 5, 9], [31, 13, 11, 15, 27, 32, 4, 8, 43, 41], [11, 33, 40, 28, 20, 30, 22, 15, 12, 21]], [[18, 24, 21, 32, 19, 40, 35, 20, 9, 38], [5, 28, 0, 19, 40, 33, 29, 10, 24, 27], [11, 22, 27, 14, 40, 9, 36, 16, 41, 5], [9, 28, 34, 15, 39, 20, 4, 33, 43, 16], [8, 25, 26, 31, 27, 20, 6, 40, 2, 30], [3, 15, 33, 28, 44, 0, 36, 12, 21, 18], [39, 17, 22, 9, 1, 33, 21, 29, 15, 36], [38, 30, 36, 40, 21, 4, 5, 22, 7, 3]]]

clss_support_imagesss_5_shot=[[[24, 30, 13, 26, 3], [16, 31, 30, 15, 22], [7, 31, 42, 1, 30], [39, 41, 6, 34, 16], [42, 5, 38, 41, 7], [2, 10, 15, 24, 38], [34, 38, 18, 5, 25], [7, 13, 26, 43, 4]], [[14, 31, 35, 11, 44], [43, 35, 29, 0, 10], [40, 42, 34, 41, 12], [21, 29, 4, 10, 19], [18, 25, 11, 5, 43], [18, 41, 2, 3, 33], [44, 1, 16, 29, 35], [36, 12, 22, 44, 29]], [[10, 26, 40, 36, 38], [41, 23, 19, 24, 6], [29, 0, 40, 32, 41], [40, 6, 0, 24, 14], [36, 35, 32, 42, 7], [13, 7, 12, 44, 19], [17, 19, 23, 8, 31], [19, 17, 0, 13, 28]], [[23, 2, 25, 26, 12], [15, 14, 24, 1, 26], [28, 30, 25, 41, 33], [39, 13, 41, 6, 35], [38, 30, 10, 12, 3], [0, 16, 27, 13, 40], [27, 42, 35, 38, 40], [14, 22, 29, 10, 37]], [[7, 34, 0, 37, 21], [28, 21, 36, 32, 33], [28, 30, 18, 8, 27], [32, 22, 36, 42, 0], [16, 11, 40, 14, 38], [36, 44, 43, 12, 38], [22, 30, 25, 4, 44], [11, 12, 26, 9, 29]], [[37, 8, 41, 16, 27], [10, 43, 42, 5, 8], [13, 11, 41, 44, 32], [36, 1, 4, 18, 24], [37, 35, 40, 12, 41], [8, 33, 15, 1, 3], [4, 19, 20, 24, 12], [21, 13, 38, 24, 9]], [[42, 30, 16, 39, 41], [13, 3, 29, 10, 25], [36, 26, 14, 27, 6], [20, 38, 33, 6, 36], [30, 26, 38, 24, 8], [8, 27, 34, 17, 43], [12, 36, 35, 25, 39], [12, 7, 22, 0, 36]], [[40, 8, 43, 38, 23], [27, 20, 14, 31, 9], [22, 12, 27, 1, 17], [6, 16, 8, 42, 3], [2, 12, 30, 44, 4], [32, 17, 21, 38, 36], [15, 25, 43, 10, 5], [2, 13, 15, 16, 43]], [[1, 26, 39, 8, 5], [12, 11, 43, 28, 40], [6, 14, 20, 2, 31], [29, 23, 1, 25, 3], [22, 42, 35, 3, 37], [14, 23, 37, 44, 30], [26, 10, 13, 32, 25], [31, 16, 35, 9, 6]], [[25, 37, 5, 10, 33], [11, 2, 23, 29, 3], [40, 17, 11, 14, 31], [39, 29, 16, 19, 6], [22, 5, 34, 36, 13], [20, 15, 17, 38, 33], [42, 44, 22, 0, 29], [4, 38, 3, 35, 19]], [[7, 29, 19, 4, 40], [38, 3, 29, 17, 14], [22, 30, 11, 17, 10], [29, 17, 9, 44, 20], [5, 21, 37, 23, 41], [23, 8, 21, 15, 43], [6, 4, 26, 38, 10], [27, 36, 2, 1, 28]], [[42, 1, 20, 12, 34], [40, 41, 43, 11, 24], [16, 34, 19, 39, 7], [24, 34, 37, 16, 35], [33, 2, 16, 7, 3], [43, 17, 24, 14, 40], [6, 3, 21, 27, 19], [16, 3, 25, 22, 38]], [[40, 30, 1, 23, 41], [18, 7, 20, 31, 21], [25, 14, 2, 28, 24], [23, 14, 32, 35, 17], [19, 17, 34, 18, 20], [4, 39, 15, 25, 44], [39, 34, 23, 13, 2], [7, 9, 19, 34, 39]], [[1, 29, 19, 21, 43], [36, 33, 23, 31, 19], [43, 26, 42, 29, 2], [36, 27, 34, 20, 43], [33, 25, 9, 16, 2], [16, 39, 40, 12, 13], [31, 13, 11, 15, 27], [11, 33, 40, 28, 20]], [[18, 24, 21, 32, 19], [5, 28, 0, 19, 40], [11, 22, 27, 14, 40], [9, 28, 34, 15, 39], [8, 25, 26, 31, 27], [3, 15, 33, 28, 44], [39, 17, 22, 9, 1], [38, 30, 36, 40, 21]]]


clss_support_imagesss_1_shot=[
   [[24], [16], [7], [39], [42], [2], [34], [7]],
   [[14], [43], [40], [21], [18], [18], [44], [36]],
   [[10], [41], [29], [40], [36], [13], [17], [19]],
   [[23], [15], [28], [39], [38], [0], [27], [14]],
   [[7], [28], [28], [32], [16], [36], [22], [11]],
   [[37], [10], [13], [36], [37], [8], [4], [21]],
   [[42], [13], [36], [20], [30], [8], [12], [12]],
   [[40], [27], [22], [6], [2], [32], [15], [2]],
   [[1], [12], [6], [29], [22], [14], [26], [31]],
   [[25], [11], [40], [39], [22], [20], [42], [4]],
   [[7], [38], [22], [29], [5], [23], [6], [27]],
   [[42], [40], [16], [24], [33], [43], [6], [16]],
   [[40], [18], [25], [23], [19], [4], [39], [7]],
   [[1], [36], [43], [36], [33], [16], [31], [11]],
   [[18], [5], [11], [9], [8], [3], [39], [38]]

]



In [66]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_1/model_B_olchiki_1-shot_dens.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [67]:


def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [68]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5052589265821088
Evaluated Accuracy: 0.5053
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.49260138949114385
Evaluated Accuracy: 0.4926
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.5585072795825865
Evaluated Accuracy: 0.5585
Testing b_a_r = 0.9468, c_a_r = 0.2837, f_a_r = 0.9970


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5423915586214925
Evaluated Accuracy: 0.5424
Testing b_a_r = 0.9695, c_a_r = 0.3459, f_a_r = 0.0163


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5552795041404579
Evaluated Accuracy: 0.5553

✅ Optimal Values:
   - b_a_r: 0.0202
   - c_a_r: 0.3343
   - f_a_r: 0.9898
📈 Highest Accuracy Achieved: 0.5619


In [69]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/8_w_1_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/8_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/8_w_1_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4566 ± 0.0955  0.5222 ± 0.0990  0.4567 ± 0.0955


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3064 ± 0.0457  0.3470 ± 0.0524  0.3066 ± 0.0458
---Accu-----pre----rec---------> 0.5619 ± 0.0783  0.6122 ± 0.0862  0.5620 ± 0.0782


In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_5/model_B_olchiki_5-shot_dens.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [71]:
def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [72]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6160490682688959
Evaluated Accuracy: 0.6160
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5864539957258698
Evaluated Accuracy: 0.5865
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.6587647157198799
Evaluated Accuracy: 0.6588
Testing b_a_r = 0.9970, c_a_r = 0.3050, f_a_r = 0.0298


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6598089987857954
Evaluated Accuracy: 0.6598
Testing b_a_r = 0.0164, c_a_r = 0.3095, f_a_r = 0.0091


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6616892234461924
Evaluated Accuracy: 0.6617

✅ Optimal Values:
   - b_a_r: 0.0268
   - c_a_r: 0.3014
   - f_a_r: 0.0070
📈 Highest Accuracy Achieved: 0.6636


In [73]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/8_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/8_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/8_w_5_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5487 ± 0.0621  0.5758 ± 0.0713  0.5488 ± 0.0620


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4208 ± 0.0692  0.4371 ± 0.0721  0.4209 ± 0.0692
---Accu-----pre----rec---------> 0.6636 ± 0.0674  0.6792 ± 0.0665  0.6637 ± 0.0673


In [74]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_10/model_B_olchiki_10-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [75]:
def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_8)):
            support_set, query_set = data_loader.__getitem__(clss_8[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_8[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_8[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [76]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=80
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6202725071201711
Evaluated Accuracy: 0.6203
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6185888516556637
Evaluated Accuracy: 0.6186
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.675038038021622
Evaluated Accuracy: 0.6750
Testing b_a_r = 0.9855, c_a_r = 0.3333, f_a_r = 0.0426


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6831520020602789
Evaluated Accuracy: 0.6832
Testing b_a_r = 0.9931, c_a_r = 0.3764, f_a_r = 0.9645


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.6740796833535899
Evaluated Accuracy: 0.6741

✅ Optimal Values:
   - b_a_r: 0.9639
   - c_a_r: 0.3431
   - f_a_r: 0.0283
📈 Highest Accuracy Achieved: 0.6855


In [77]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/8_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/8_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/8_w_10_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5693 ± 0.0638  0.5906 ± 0.0711  0.5693 ± 0.0637


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.5433 ± 0.0685  0.5487 ± 0.0780  0.5435 ± 0.0686
---Accu-----pre----rec---------> 0.6855 ± 0.0603  0.6942 ± 0.0589  0.6856 ± 0.0602


In [ ]:
###############################################   12 way 1 shot

In [78]:
clss_support_imagesss_10_shot=[[[4, 37, 38, 22, 13, 16, 5, 43, 27, 21], [23, 20, 4, 37, 0, 3, 34, 29, 6, 43], [38, 41, 12, 27, 43, 24, 22, 23, 8, 26], [41, 37, 8, 0, 20, 14, 21, 39, 7, 35], [43, 33, 21, 37, 25, 32, 34, 9, 11, 28], [25, 13, 27, 39, 1, 16, 28, 15, 38, 41], [1, 44, 32, 24, 12, 3, 40, 26, 33, 39], [37, 32, 30, 26, 39, 24, 7, 17, 36, 35], [40, 37, 16, 5, 25, 1, 4, 30, 38, 33], [8, 44, 10, 31, 3, 14, 0, 16, 15, 21], [27, 15, 31, 14, 1, 6, 22, 39, 21, 35], [35, 31, 12, 8, 22, 2, 30, 37, 21, 36]], [[20, 4, 39, 43, 12, 31, 14, 8, 30, 38], [5, 9, 17, 41, 37, 1, 34, 32, 21, 25], [15, 5, 35, 41, 42, 34, 4, 1, 37, 22], [36, 21, 9, 26, 5, 32, 24, 35, 11, 7], [40, 2, 30, 43, 14, 32, 0, 13, 12, 4], [29, 42, 32, 34, 8, 26, 3, 7, 33, 25], [22, 36, 25, 17, 28, 20, 39, 38, 27, 9], [1, 13, 20, 9, 40, 44, 35, 37, 10, 12], [10, 39, 27, 28, 26, 1, 23, 12, 40, 18], [22, 4, 37, 17, 18, 0, 44, 39, 24, 33], [35, 36, 13, 28, 26, 6, 0, 39, 43, 7], [42, 20, 25, 11, 8, 16, 22, 35, 31, 13]], [[21, 6, 0, 43, 34, 29, 33, 19, 2, 27], [7, 6, 38, 18, 11, 20, 30, 1, 33, 21], [25, 24, 32, 35, 28, 27, 2, 8, 31, 9], [24, 0, 28, 5, 6, 22, 32, 3, 2, 12], [17, 19, 32, 13, 0, 39, 25, 37, 15, 9], [14, 40, 13, 32, 33, 28, 5, 39, 16, 12], [31, 7, 42, 25, 39, 19, 27, 18, 15, 32], [27, 33, 28, 31, 38, 22, 42, 20, 19, 34], [40, 27, 43, 42, 15, 4, 35, 34, 20, 21], [12, 41, 27, 36, 9, 16, 10, 2, 43, 32], [42, 30, 41, 34, 7, 20, 9, 28, 36, 40], [13, 42, 36, 41, 44, 5, 24, 7, 9, 4]], [[33, 28, 15, 41, 3, 24, 4, 32, 39, 8], [29, 36, 5, 32, 34, 7, 15, 1, 10, 44], [6, 25, 21, 7, 0, 8, 40, 14, 12, 41], [37, 27, 10, 40, 12, 42, 7, 24, 21, 44], [43, 18, 12, 42, 25, 26, 32, 21, 13, 29], [10, 28, 27, 8, 32, 13, 16, 3, 30, 22], [6, 31, 27, 7, 1, 20, 30, 25, 38, 18], [44, 30, 32, 25, 13, 20, 16, 4, 9, 31], [17, 21, 6, 41, 20, 36, 43, 1, 9, 44], [12, 25, 18, 35, 44, 33, 10, 41, 21, 37], [21, 12, 29, 44, 30, 5, 34, 40, 42, 15], [10, 42, 41, 9, 26, 25, 19, 27, 37, 34]], [[1, 39, 41, 30, 6, 3, 10, 14, 25, 16], [6, 9, 28, 25, 5, 20, 4, 43, 0, 24], [21, 42, 7, 18, 2, 28, 9, 32, 1, 20], [43, 5, 37, 13, 25, 31, 12, 19, 26, 33], [13, 35, 29, 44, 31, 12, 34, 10, 43, 33], [17, 21, 20, 34, 8, 6, 35, 37, 1, 33], [38, 32, 13, 23, 19, 24, 2, 31, 43, 36], [35, 37, 12, 7, 24, 4, 38, 21, 16, 8], [10, 15, 32, 38, 33, 24, 18, 39, 1, 34], [4, 17, 8, 5, 43, 38, 22, 3, 13, 11], [17, 2, 1, 32, 34, 39, 7, 13, 16, 41], [28, 33, 16, 43, 29, 5, 3, 14, 13, 26]], [[25, 9, 3, 12, 33, 30, 2, 21, 43, 41], [37, 4, 27, 39, 2, 16, 1, 11, 10, 35], [24, 22, 10, 7, 18, 40, 31, 25, 14, 32], [3, 35, 26, 4, 21, 5, 30, 8, 38, 29], [12, 14, 24, 0, 16, 3, 43, 5, 41, 9], [20, 4, 27, 0, 2, 36, 12, 32, 29, 40], [21, 7, 24, 8, 36, 29, 17, 35, 14, 11], [40, 17, 38, 29, 2, 14, 36, 22, 37, 35], [30, 16, 1, 36, 18, 13, 28, 43, 39, 5], [11, 0, 19, 14, 33, 40, 30, 20, 13, 34], [16, 20, 5, 9, 37, 25, 23, 10, 17, 14], [20, 7, 44, 30, 29, 4, 39, 28, 26, 22]], [[34, 22, 31, 32, 11, 23, 42, 20, 15, 5], [1, 3, 36, 23, 43, 34, 16, 14, 21, 40], [20, 6, 42, 23, 25, 8, 31, 22, 7, 26], [37, 27, 35, 8, 42, 32, 16, 43, 36, 24], [44, 40, 20, 1, 14, 13, 3, 22, 33, 23], [18, 27, 34, 7, 11, 33, 6, 30, 43, 31], [44, 2, 0, 31, 13, 16, 9, 11, 20, 29], [25, 22, 28, 3, 4, 18, 31, 30, 23, 34], [38, 2, 18, 33, 17, 12, 32, 43, 34, 5], [13, 23, 19, 5, 36, 35, 1, 34, 32, 6], [44, 15, 25, 2, 43, 39, 24, 30, 42, 31], [20, 31, 30, 38, 29, 35, 16, 9, 26, 22]], [[42, 5, 41, 16, 24, 28, 26, 36, 17, 20], [39, 27, 33, 22, 37, 9, 11, 31, 38, 14], [44, 3, 17, 4, 35, 39, 10, 21, 18, 32], [26, 3, 41, 12, 38, 24, 25, 10, 0, 37], [29, 24, 32, 0, 36, 25, 19, 6, 3, 10], [31, 5, 21, 26, 14, 39, 22, 18, 15, 27], [3, 42, 26, 28, 41, 5, 10, 1, 15, 32], [41, 44, 12, 27, 21, 33, 38, 9, 13, 0], [9, 32, 22, 6, 20, 16, 7, 41, 24, 8], [30, 37, 11, 35, 23, 1, 10, 41, 28, 6], [1, 8, 12, 0, 4, 11, 16, 40, 2, 22], [38, 31, 2, 28, 33, 23, 44, 16, 43, 19]], [[41, 13, 0, 2, 42, 30, 43, 10, 14, 37], [38, 33, 9, 12, 21, 1, 8, 23, 13, 37], [33, 42, 24, 5, 36, 13, 11, 16, 19, 2], [29, 12, 31, 10, 32, 7, 41, 27, 33, 39], [3, 13, 41, 16, 10, 28, 42, 12, 22, 14], [5, 35, 28, 0, 33, 22, 40, 30, 36, 24], [16, 6, 38, 1, 26, 24, 39, 18, 7, 37], [37, 18, 12, 27, 24, 11, 29, 4, 21, 30], [18, 30, 0, 19, 34, 43, 22, 33, 3, 36], [30, 44, 38, 4, 40, 0, 32, 19, 23, 27], [26, 7, 22, 41, 4, 9, 5, 29, 32, 43], [44, 40, 5, 10, 11, 6, 25, 27, 23, 34]], [[28, 37, 1, 16, 19, 34, 3, 30, 44, 2], [10, 1, 8, 7, 12, 34, 20, 11, 32, 25], [2, 5, 16, 37, 32, 17, 6, 7, 28, 4], [15, 0, 12, 9, 19, 41, 1, 38, 28, 31], [15, 33, 17, 41, 9, 39, 22, 29, 10, 14], [44, 8, 41, 39, 4, 19, 29, 3, 12, 30], [33, 36, 26, 3, 9, 7, 21, 31, 8, 34], [36, 9, 42, 40, 33, 30, 6, 17, 1, 3], [21, 40, 39, 28, 6, 27, 18, 7, 20, 36], [42, 7, 44, 35, 28, 6, 8, 30, 1, 33], [11, 35, 26, 6, 44, 32, 8, 39, 13, 27], [10, 40, 28, 20, 22, 8, 29, 2, 33, 21]], [[20, 15, 39, 16, 5, 42, 24, 3, 23, 12], [29, 40, 21, 44, 36, 16, 33, 38, 15, 31], [19, 4, 9, 43, 11, 3, 22, 27, 5, 24], [36, 15, 20, 37, 25, 41, 16, 17, 21, 35], [10, 16, 41, 29, 14, 22, 37, 38, 31, 9], [9, 13, 21, 11, 4, 39, 5, 28, 37, 16], [41, 16, 21, 3, 24, 32, 19, 1, 34, 44], [3, 20, 0, 30, 41, 5, 4, 6, 28, 9], [32, 20, 27, 34, 8, 31, 0, 17, 18, 13], [21, 2, 15, 20, 32, 38, 12, 33, 17, 23], [33, 36, 31, 34, 22, 6, 12, 11, 20, 29], [13, 27, 32, 34, 26, 44, 17, 8, 21, 0]], [[34, 31, 35, 29, 42, 13, 41, 32, 37, 43], [17, 9, 0, 1, 26, 44, 16, 25, 12, 31], [23, 27, 6, 25, 8, 37, 15, 43, 30, 32], [1, 5, 16, 21, 15, 44, 35, 4, 28, 20], [23, 44, 35, 4, 12, 22, 34, 29, 26, 37], [17, 9, 20, 44, 21, 41, 13, 3, 35, 5], [17, 37, 41, 24, 28, 14, 16, 23, 44, 30], [26, 25, 9, 5, 44, 35, 16, 2, 6, 15], [42, 39, 0, 36, 11, 12, 37, 4, 32, 19], [35, 21, 31, 44, 10, 13, 0, 34, 29, 7], [14, 8, 6, 13, 5, 28, 35, 39, 1, 38], [2, 18, 26, 34, 17, 29, 24, 0, 11, 7]], [[8, 32, 11, 36, 17, 1, 24, 21, 6, 20], [40, 23, 13, 35, 37, 3, 22, 24, 21, 28], [3, 35, 8, 15, 25, 28, 37, 20, 5, 43], [41, 44, 43, 39, 29, 1, 7, 3, 6, 13], [40, 21, 19, 23, 7, 10, 14, 32, 24, 15], [33, 23, 21, 35, 19, 39, 28, 13, 25, 0], [5, 1, 40, 35, 11, 13, 34, 20, 10, 33], [24, 29, 27, 39, 44, 30, 18, 26, 1, 33], [0, 16, 34, 5, 8, 2, 44, 20, 38, 42], [33, 13, 20, 34, 31, 41, 6, 17, 0, 26], [7, 29, 28, 33, 34, 21, 26, 2, 40, 36], [16, 28, 44, 24, 29, 42, 25, 6, 10, 11]], [[7, 1, 13, 3, 41, 10, 11, 12, 25, 27], [15, 35, 26, 29, 21, 10, 27, 20, 18, 4], [28, 20, 23, 34, 39, 36, 0, 31, 15, 18], [24, 27, 33, 11, 2, 18, 42, 14, 26, 9], [9, 30, 35, 37, 24, 19, 44, 8, 10, 33], [7, 19, 25, 36, 26, 4, 16, 8, 15, 12], [24, 9, 15, 44, 22, 8, 19, 5, 27, 20], [34, 15, 35, 24, 27, 10, 25, 19, 12, 30], [27, 28, 12, 34, 0, 9, 1, 33, 42, 2], [7, 8, 36, 42, 25, 23, 17, 31, 33, 41], [15, 31, 0, 6, 25, 20, 32, 12, 35, 43], [6, 14, 28, 0, 26, 8, 9, 35, 5, 1]], [[31, 37, 25, 10, 28, 14, 34, 21, 39, 7], [19, 25, 7, 29, 18, 4, 32, 40, 42, 30], [44, 26, 11, 32, 19, 8, 16, 37, 29, 40], [13, 44, 38, 33, 0, 40, 22, 15, 1, 32], [34, 2, 28, 14, 23, 36, 39, 38, 5, 13], [7, 0, 26, 40, 25, 43, 37, 35, 15, 11], [35, 8, 30, 20, 43, 32, 29, 0, 12, 18], [33, 20, 25, 36, 7, 14, 21, 42, 37, 30], [41, 37, 13, 25, 38, 20, 29, 15, 30, 17], [42, 4, 14, 43, 12, 0, 13, 11, 40, 29], [41, 23, 22, 40, 21, 12, 0, 37, 16, 44], [41, 43, 2, 32, 16, 11, 8, 4, 1, 17]]]



clss_support_imagesss_5_shot=[[[4, 37, 38, 22, 13], [23, 20, 4, 37, 0], [38, 41, 12, 27, 43], [41, 37, 8, 0, 20], [43, 33, 21, 37, 25], [25, 13, 27, 39, 1], [1, 44, 32, 24, 12], [37, 32, 30, 26, 39], [40, 37, 16, 5, 25], [8, 44, 10, 31, 3], [27, 15, 31, 14, 1], [35, 31, 12, 8, 22]], [[20, 4, 39, 43, 12], [5, 9, 17, 41, 37], [15, 5, 35, 41, 42], [36, 21, 9, 26, 5], [40, 2, 30, 43, 14], [29, 42, 32, 34, 8], [22, 36, 25, 17, 28], [1, 13, 20, 9, 40], [10, 39, 27, 28, 26], [22, 4, 37, 17, 18], [35, 36, 13, 28, 26], [42, 20, 25, 11, 8]], [[21, 6, 0, 43, 34], [7, 6, 38, 18, 11], [25, 24, 32, 35, 28], [24, 0, 28, 5, 6], [17, 19, 32, 13, 0], [14, 40, 13, 32, 33], [31, 7, 42, 25, 39], [27, 33, 28, 31, 38], [40, 27, 43, 42, 15], [12, 41, 27, 36, 9], [42, 30, 41, 34, 7], [13, 42, 36, 41, 44]], [[33, 28, 15, 41, 3], [29, 36, 5, 32, 34], [6, 25, 21, 7, 0], [37, 27, 10, 40, 12], [43, 18, 12, 42, 25], [10, 28, 27, 8, 32], [6, 31, 27, 7, 1], [44, 30, 32, 25, 13], [17, 21, 6, 41, 20], [12, 25, 18, 35, 44], [21, 12, 29, 44, 30], [10, 42, 41, 9, 26]], [[1, 39, 41, 30, 6], [6, 9, 28, 25, 5], [21, 42, 7, 18, 2], [43, 5, 37, 13, 25], [13, 35, 29, 44, 31], [17, 21, 20, 34, 8], [38, 32, 13, 23, 19], [35, 37, 12, 7, 24], [10, 15, 32, 38, 33], [4, 17, 8, 5, 43], [17, 2, 1, 32, 34], [28, 33, 16, 43, 29]], [[25, 9, 3, 12, 33], [37, 4, 27, 39, 2], [24, 22, 10, 7, 18], [3, 35, 26, 4, 21], [12, 14, 24, 0, 16], [20, 4, 27, 0, 2], [21, 7, 24, 8, 36], [40, 17, 38, 29, 2], [30, 16, 1, 36, 18], [11, 0, 19, 14, 33], [16, 20, 5, 9, 37], [20, 7, 44, 30, 29]], [[34, 22, 31, 32, 11], [1, 3, 36, 23, 43], [20, 6, 42, 23, 25], [37, 27, 35, 8, 42], [44, 40, 20, 1, 14], [18, 27, 34, 7, 11], [44, 2, 0, 31, 13], [25, 22, 28, 3, 4], [38, 2, 18, 33, 17], [13, 23, 19, 5, 36], [44, 15, 25, 2, 43], [20, 31, 30, 38, 29]], [[42, 5, 41, 16, 24], [39, 27, 33, 22, 37], [44, 3, 17, 4, 35], [26, 3, 41, 12, 38], [29, 24, 32, 0, 36], [31, 5, 21, 26, 14], [3, 42, 26, 28, 41], [41, 44, 12, 27, 21], [9, 32, 22, 6, 20], [30, 37, 11, 35, 23], [1, 8, 12, 0, 4], [38, 31, 2, 28, 33]], [[41, 13, 0, 2, 42], [38, 33, 9, 12, 21], [33, 42, 24, 5, 36], [29, 12, 31, 10, 32], [3, 13, 41, 16, 10], [5, 35, 28, 0, 33], [16, 6, 38, 1, 26], [37, 18, 12, 27, 24], [18, 30, 0, 19, 34], [30, 44, 38, 4, 40], [26, 7, 22, 41, 4], [44, 40, 5, 10, 11]], [[28, 37, 1, 16, 19], [10, 1, 8, 7, 12], [2, 5, 16, 37, 32], [15, 0, 12, 9, 19], [15, 33, 17, 41, 9], [44, 8, 41, 39, 4], [33, 36, 26, 3, 9], [36, 9, 42, 40, 33], [21, 40, 39, 28, 6], [42, 7, 44, 35, 28], [11, 35, 26, 6, 44], [10, 40, 28, 20, 22]], [[20, 15, 39, 16, 5], [29, 40, 21, 44, 36], [19, 4, 9, 43, 11], [36, 15, 20, 37, 25], [10, 16, 41, 29, 14], [9, 13, 21, 11, 4], [41, 16, 21, 3, 24], [3, 20, 0, 30, 41], [32, 20, 27, 34, 8], [21, 2, 15, 20, 32], [33, 36, 31, 34, 22], [13, 27, 32, 34, 26]], [[34, 31, 35, 29, 42], [17, 9, 0, 1, 26], [23, 27, 6, 25, 8], [1, 5, 16, 21, 15], [23, 44, 35, 4, 12], [17, 9, 20, 44, 21], [17, 37, 41, 24, 28], [26, 25, 9, 5, 44], [42, 39, 0, 36, 11], [35, 21, 31, 44, 10], [14, 8, 6, 13, 5], [2, 18, 26, 34, 17]], [[8, 32, 11, 36, 17], [40, 23, 13, 35, 37], [3, 35, 8, 15, 25], [41, 44, 43, 39, 29], [40, 21, 19, 23, 7], [33, 23, 21, 35, 19], [5, 1, 40, 35, 11], [24, 29, 27, 39, 44], [0, 16, 34, 5, 8], [33, 13, 20, 34, 31], [7, 29, 28, 33, 34], [16, 28, 44, 24, 29]], [[7, 1, 13, 3, 41], [15, 35, 26, 29, 21], [28, 20, 23, 34, 39], [24, 27, 33, 11, 2], [9, 30, 35, 37, 24], [7, 19, 25, 36, 26], [24, 9, 15, 44, 22], [34, 15, 35, 24, 27], [27, 28, 12, 34, 0], [7, 8, 36, 42, 25], [15, 31, 0, 6, 25], [6, 14, 28, 0, 26]], [[31, 37, 25, 10, 28], [19, 25, 7, 29, 18], [44, 26, 11, 32, 19], [13, 44, 38, 33, 0], [34, 2, 28, 14, 23], [7, 0, 26, 40, 25], [35, 8, 30, 20, 43], [33, 20, 25, 36, 7], [41, 37, 13, 25, 38], [42, 4, 14, 43, 12], [41, 23, 22, 40, 21], [41, 43, 2, 32, 16]]]


clss_12=[['assamese1_PAC', 'assamese1_KTA', 'assamese1_A', 'assamese1_JA', 'assamese1_DXA', 'assamese1_KHA', 'assamese1_OI', 'assamese1_OU', 'assamese1_E', 'assamese1_CAY', 'assamese1_DHRA', 'assamese1_XAT'],
         ['assamese1_CCA', 'assamese1_PA', 'assamese1_DHA', 'assamese1_A', 'assamese1_PAC', 'assamese1_GA', 'assamese1_MTA', 'assamese1_CA', 'assamese1_EE', 'assamese1_CAY', 'assamese1_NIYA', 'assamese1_AA'],
        ['assamese1_OI', 'assamese1_EK', 'assamese1_MNA', 'assamese1_GA', 'assamese1_TINI', 'assamese1_CARI', 'assamese1_KHA', 'assamese1_DXA', 'assamese1_ANSR', 'assamese1_CA', 'assamese1_KTA', 'assamese1_AJA'],
         ['assamese1_ANSR', 'assamese1_HA', 'assamese1_AE', 'assamese1_AA', 'assamese1_MDHA', 'assamese1_DHA', 'assamese1_CAY', 'assamese1_O', 'assamese1_ATH', 'assamese1_KHA', 'assamese1_BXG', 'assamese1_NAA'],
          ['assamese1_E', 'assamese1_OU', 'assamese1_TINI', 'assamese1_GHA', 'assamese1_CCA', 'assamese1_NA', 'assamese1_NG', 'assamese1_MNA', 'assamese1_PAC', 'assamese1_AE', 'assamese1_TXA', 'assamese1_BXG'],
          ['assamese1_TA', 'assamese1_MDHA', 'assamese1_DA', 'assamese1_TINI', 'assamese1_KA', 'assamese1_EE', 'assamese1_GHA', 'assamese1_OI', 'assamese1_AA', 'assamese1_DXA', 'assamese1_DHRA', 'assamese1_CCA'],
          ['assamese1_SUNYA', 'assamese1_KHA', 'assamese1_ATH', 'assamese1_AA', 'assamese1_CAY', 'assamese1_DUI', 'assamese1_NG', 'assamese1_MA', 'assamese1_CA', 'assamese1_MNA', 'assamese1_REE', 'assamese1_KHYA'],
         ['assamese1_KHA', 'assamese1_NAA', 'assamese1_OU', 'assamese1_DRA', 'assamese1_DXA', 'assamese1_CBN', 'assamese1_SUNYA', 'assamese1_BXG', 'assamese1_AYA', 'assamese1_BHA', 'assamese1_JHA', 'assamese1_AE'],
        ['assamese1_CAY', 'assamese1_E', 'assamese1_XAT', 'assamese1_KTA', 'assamese1_JHA', 'assamese1_PHA', 'assamese1_KHA', 'assamese1_TXA', 'assamese1_UU', 'assamese1_CA', 'assamese1_DUI', 'assamese1_AJA'],
         ['assamese1_EK', 'assamese1_CA', 'assamese1_TINI', 'assamese1_DRA', 'assamese1_DXA', 'assamese1_CAY', 'assamese1_JA', 'assamese1_HA', 'assamese1_CCA', 'assamese1_O', 'assamese1_NIYA', 'assamese1_ATH'],
        ['assamese1_THA', 'assamese1_DRA', 'assamese1_DHRA', 'assamese1_BA', 'assamese1_BHA', 'assamese1_KTA', 'assamese1_NAA', 'assamese1_PAC', 'assamese1_E', 'assamese1_CAY', 'assamese1_JHA', 'assamese1_ANSR'],
         ['assamese1_BXG', 'assamese1_REE', 'assamese1_CA', 'assamese1_TR', 'assamese1_NAA', 'assamese1_THA', 'assamese1_LA', 'assamese1_U', 'assamese1_CCA', 'assamese1_DUI', 'assamese1_MNA', 'assamese1_MDA'],
        ['assamese1_AE', 'assamese1_MXA', 'assamese1_O', 'assamese1_TR', 'assamese1_E', 'assamese1_PA', 'assamese1_CCA', 'assamese1_GA', 'assamese1_MTHA', 'assamese1_ANSR', 'assamese1_MDHA', 'assamese1_HA'],
         ['assamese1_EK', 'assamese1_CAY', 'assamese1_U', 'assamese1_CCA', 'assamese1_BXG', 'assamese1_ANSR', 'assamese1_NIYA', 'assamese1_PHA', 'assamese1_GA', 'assamese1_DHA', 'assamese1_A', 'assamese1_MDHA'],
         ['assamese1_NG', 'assamese1_NA', 'assamese1_NIYA', 'assamese1_ATH', 'assamese1_MNA', 'assamese1_DXA', 'assamese1_O', 'assamese1_MTA', 'assamese1_CAY', 'assamese1_SUNYA', 'assamese1_AE', 'assamese1_XAT'],
    ]
clss_support_imagesss_1_shot=[
    [[4], [23], [38], [41], [43], [25], [1], [37], [40], [8], [27], [35]],
    [[20], [5], [15], [36], [40], [29], [22], [1], [10], [22], [35], [42]],
    [[21], [7], [25], [24], [17], [14], [31], [27], [40], [12], [42], [13]],
    [[33], [29], [6], [37], [43], [10], [6], [44], [17], [12], [21], [10]],
    [[1], [6], [21], [43], [13], [17], [38], [35], [10], [4], [17], [28]],
    [[25], [37], [24], [3], [12], [20], [21], [40], [30], [11], [16], [20]],
    [[34], [1], [20], [37], [44], [18], [44], [25], [38], [13], [44], [20]],
    [[42], [39], [44], [26], [29], [31], [3], [41], [9], [30], [1], [38]],
     [[41], [38], [33], [29], [3], [5], [16], [37], [18], [30], [26], [44]],
    [[28], [10], [2], [15], [15], [44], [33], [36], [21], [42], [11], [10]],
    [[20], [29], [19], [36], [10], [9], [41], [3], [32], [21], [33], [13]],
     [[34], [17], [23], [1], [23], [17], [17], [26], [42], [35], [14], [2]],
    [[8], [40], [3], [41], [40], [33], [5], [24], [0], [33], [7], [16]],
    [[7], [15], [28], [24], [9], [7], [24], [34], [27], [7], [15], [6]],
    [[31], [19], [44], [13], [34], [7], [35], [33], [41], [42], [41], [41]]
 ]














In [79]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_1/model_B_olchiki_1-shot_dens.pth',map_location=torch.device('cpu')))
torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [80]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_1_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_1_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [81]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.34690991879410116
Evaluated Accuracy: 0.3469
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.35537125793432583
Evaluated Accuracy: 0.3554
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.40714690254208236
Evaluated Accuracy: 0.4071
Testing b_a_r = 0.9881, c_a_r = 0.3180, f_a_r = 0.9645


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.4041106306316882
Evaluated Accuracy: 0.4041
Testing b_a_r = 0.0091, c_a_r = 0.0218, f_a_r = 0.4557


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.33727450320618513
Evaluated Accuracy: 0.3373

✅ Optimal Values:
   - b_a_r: 0.0180
   - c_a_r: 0.3408
   - f_a_r: 0.9957
📈 Highest Accuracy Achieved: 0.4137


In [82]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/12_w_1_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/12_w_1_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/12_w_1_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3324 ± 0.0425  0.3585 ± 0.0561  0.3325 ± 0.0424


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.2675 ± 0.0321  0.2821 ± 0.0391  0.2675 ± 0.0321
---Accu-----pre----rec---------> 0.4137 ± 0.0703  0.4434 ± 0.0560  0.4136 ± 0.0705


In [83]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_5/model_B_olchiki_5-shot_dens.pth',map_location=torch.device('cpu')))


torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [84]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_5_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_5_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [85]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.47856280565085646
Evaluated Accuracy: 0.4786
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.45842211088891155
Evaluated Accuracy: 0.4584
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.5405166046401461
Evaluated Accuracy: 0.5405
Testing b_a_r = 0.9821, c_a_r = 0.3603, f_a_r = 0.9953


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.5349485893966962
Evaluated Accuracy: 0.5349
Testing b_a_r = 0.0041, c_a_r = 0.4768, f_a_r = 0.1770


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.4299417469861167
Evaluated Accuracy: 0.4299

✅ Optimal Values:
   - b_a_r: 0.0273
   - c_a_r: 0.3082
   - f_a_r: 0.9889
📈 Highest Accuracy Achieved: 0.5407


In [86]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/12_w_5_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/12_w_5_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/12_w_5_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4333 ± 0.0572  0.4498 ± 0.0599  0.4332 ± 0.0573


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.3456 ± 0.0422  0.3614 ± 0.0555  0.3456 ± 0.0422
---Accu-----pre----rec---------> 0.5407 ± 0.0606  0.5433 ± 0.0625  0.5405 ± 0.0607


In [87]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



!rm model_mu_path.pt
!rm model_own_path.pt

convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/modelrdnet.pth',map_location=torch.device('cpu')))


convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
model_own_1_shot = PrototypicalNetworks_dynamic_query(convolutional_network_with_dropout).to(device)
model_own_1_shot.load_state_dict(torch.load('/home/asufian/Desktop/output_olchiki/code/asamese_sce/denssecondary/model_10/model_B_olchiki_10-shot_dens.pth',map_location=torch.device('cpu')))

torch.save(model_own_1_shot.state_dict(), 'model_mu_path.pt')
torch.save(model_own.state_dict(), 'model_own_path.pt')

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


In [88]:

def evaluate2(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlal=[]
    ttlac=0
    precision=[]
    recall=[]
    
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

           
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            ttlal.append(accuracy)
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            precision_overall, recall_overall, f1ss= calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)
            precision.append(precision_overall)
            recall.append(recall_overall)
    print(f"---Accu-----pre----rec---------> {np.mean(ttlal):.4f} ± {np.std(ttlal, ddof=1):.4f}  "
      f"{np.mean(precision):.4f} ± {np.std(precision, ddof=1):.4f}  "
      f"{np.mean(recall):.4f} ± {np.std(recall, ddof=1):.4f}")

    # print("---Accu-----pre----rec---------->",np.mean(ttlal),'±',np.std(ttlal, ddof=1), np.mean(precision),'±',np.std(precision, ddof=1),np.mean(precision),'±',np.std(precision, ddof=1),)
# %%%%%%

def evaluate3(fname,data_loader, model, criterion=nn.CrossEntropyLoss()):
    total_predictions = 0
    correct_predictions = 0
    total_loss = 0.0
    accuracy = 0
    datta = ""
    model.eval()
    tlv_cls=0
    ttla=0
    ttlac=0
    with torch.no_grad():
        for episode in range(len(clss_12)):
            support_set, query_set = data_loader.__getitem__(clss_12[tlv_cls] , clss_support_imagesss_10_shot[tlv_cls])

            
            clssa=str(clss_12[tlv_cls])
            clss=str(clssa).replace(',',' ').replace('\'',' ').replace('\'',' ')
            msg=str(clss_support_imagesss_10_shot[tlv_cls])
            str(clss_12[tlv_cls])

            tlv_cls+=1
            # print(len(support_set),len(query_set))
            support_set = [(torch.stack(images).to(device), label[0]) for images, label in support_set]
            query_set = [(torch.stack(images).to(device), label) for images, label in query_set]

            support_images, support_labels = zip(*support_set)

            query_images, query_labels = zip(*query_set)

            classification_scores = model(support_images, support_labels, query_images)
            cortt = 0
            totl = 0
            confusion_mat=None
            all_predicted_labels = []
            all_actual_labels = []
            # print(len(classification_scores),len(query_labels))
            for ei in range(len(classification_scores)):
                classification_scores_each_class = classification_scores[ei]
                # print(classification_scores_each_class)
                predicted_labels_eachclas = torch.argmax(classification_scores_each_class, dim=1)
                pp = predicted_labels_eachclas.tolist()
                act = query_labels[ei]
                all_predicted_labels.extend(pp)
                all_actual_labels.extend(act)
                for iiii in range(len(pp)):
                    if pp[iiii] == act[iiii]:
                        cortt = cortt + 1
                totl = totl + len(pp)
                # print(len(act))
            accuracy = cortt / totl
            ttla+=1
            ttlac+=accuracy
            # print(cortt , totl,"Validation Accuracy:              ", cortt / totl)
            # confusion_mat = confusion_matrix(all_actual_labels, all_predicted_labels)
            # print("Confusion Matrix:")
            # print(confusion_mat)

            # fname='./own35_mapi_5_way_1_shot'

            # calculate_metrics_get_per(msg,confusion_mat,acciuracy=cortt / totl,cls=clss,flnm=fname)

    print("---------------------->",ttlac/ttla)
    return ttlac/ttla

In [89]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from skopt import gp_minimize
from skopt.space import Real
from skopt.utils import use_named_args
from skopt.plots import plot_convergence

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Define the search space for b_a_r, c_a_r, and f_a_r
search_space = [
    Real(0.0, 1.0, name="b_a_r"),
    Real(0.0, 1.0, name="c_a_r"),
    Real(0.0, 1.0, name="f_a_r")
]

# Define the objective function for Bayesian Optimization
@use_named_args(search_space)
def objective(b_a_r, c_a_r, f_a_r):
    """
    Objective function to optimize the adaptation ratios.
    It applies adaptation, evaluates the model, and returns the negative accuracy.
    """
    accuracy = get_model_accuracy(b_a_r, c_a_r, f_a_r)
    print(f"Evaluated Accuracy: {accuracy:.4f}")  # Debugging Output
    
    return -accuracy  # We minimize, so return negative accuracy

# Perform Bayesian Optimization with additional settings
search_iteration=50
res = gp_minimize(
    func=objective,
    dimensions=search_space,
    n_calls=search_iteration,               # Number of evaluations
    n_initial_points=5,       # Initial random explorations before GP starts
    acq_func="EI",            # Acquisition function: Expected Improvement
    random_state=42,          # Ensure reproducibility
    n_jobs=-1,                # Parallel execution for faster optimization
)

# Extract optimal results
optimal_b_a_r, optimal_c_a_r, optimal_f_a_r = res.x
best_accuracy = -res.fun

print(f"\n✅ Optimal Values:")
print(f"   - b_a_r: {optimal_b_a_r:.4f}")
print(f"   - c_a_r: {optimal_c_a_r:.4f}")
print(f"   - f_a_r: {optimal_f_a_r:.4f}")
print(f"📈 Highest Accuracy Achieved: {best_accuracy:.4f}")




Testing b_a_r = 0.7965, c_a_r = 0.1834, f_a_r = 0.7797


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.49853138632019955
Evaluated Accuracy: 0.4985
Testing b_a_r = 0.5969, c_a_r = 0.4458, f_a_r = 0.1000


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.48662736419571945
Evaluated Accuracy: 0.4866
Testing b_a_r = 0.4592, c_a_r = 0.3337, f_a_r = 0.1429


...

----------------------> 0.5577480683553603
Evaluated Accuracy: 0.5577
Testing b_a_r = 0.0163, c_a_r = 0.5004, f_a_r = 0.5321


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.41294009494907563
Evaluated Accuracy: 0.4129
Testing b_a_r = 0.9998, c_a_r = 0.8621, f_a_r = 0.9680


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0


----------------------> 0.3127750088226887
Evaluated Accuracy: 0.3128

✅ Optimal Values:
   - b_a_r: 0.9859
   - c_a_r: 0.3350
   - f_a_r: 0.9237
📈 Highest Accuracy Achieved: 0.5642


In [90]:
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz = convolutional_network_with_dropout
M3 = PrototypicalNetworks_dynamic_query(xyz).to(device)
M3.load_state_dict(torch.load('model_own_path.pt'))
evaluate2('/home/asufian/Desktop/output_assamese/Densnet_resluts/base/12_w_10_s_f', test_loader, M3, criterion)
# Load Model M2
convolutional_network_with_dropout = DenseNet169WithDropout(modell='densenet169',pretrained=False,dr=0.0)
convolutional_network_with_dropout.fc = nn.Flatten()
xyz2 = convolutional_network_with_dropout
M2 = PrototypicalNetworks_dynamic_query(xyz2).to(device)
M2.load_state_dict(torch.load('model_mu_path.pt'))
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/secondary/12_w_10_s_f", test_loader, M2, criterion)
# Update Model M3's weights using a weighted combination of Model M3 and Model M2
update_model_weights2(M3, M2, conv_ratio= optimal_c_a_r , fc_ratio=optimal_f_a_r , bias_ratio=optimal_b_a_r)

# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
evaluate2("/home/asufian/Desktop/output_assamese/Densnet_resluts/amul/12_w_10_s_f", test_loader, M3, criterion)


# Assuming you have the evaluate2 function defined
# Evaluate the updated model (M3) using evaluate2 function with some arguments
# evaluate3(pp+'/_olchiki_5-wau-1-shot_matchine_unlearning_own', test_loader, M3, criterion)

Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4575 ± 0.0500  0.4769 ± 0.0489  0.4574 ± 0.0501


Using cache found in /home/asufian/.cache/torch/hub/pytorch_vision_v0.10.0
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/asufian/.local/share/pipx/venvs/notebook/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


---Accu-----pre----rec---------> 0.4279 ± 0.0409  0.4268 ± 0.0361  0.4278 ± 0.0410
---Accu-----pre----rec---------> 0.5642 ± 0.0575  0.5679 ± 0.0590  0.5642 ± 0.0576
